In [70]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
import tensorflow as tf
#import random
import numpy as np
import random
from ast import literal_eval
from datasets import Dataset
from transformers import AutoTokenizer

## Data Cleaning

In [71]:
# remove punctuations 
def custom_standardization(input_data):
#     lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(input_data,'[%s]' % re.escape(string.punctuation), '').numpy().decode('utf-8')

def remove_alphanumeric(text):
    return re.sub('\w*\d\w*', '', text)
def remove_extra_spaces(text):
    cleaned_text =' '.join(text.split())
    return ''.join(char for char in cleaned_text if (not char.isdigit()) and (char in string.printable))

In [72]:
# tsv_file_path = "your_file.tsv"
columns=["unsplitted", "splitted"]
wikiDF = pd.read_csv('/kaggle/input/wikisplit2/wikisplit2/wiki-split-master/train.tsv/train.tsv', sep='\t',names=columns)
wikiDF=wikiDF.head(10000)
wikiDF.head()

,unsplitted,splitted
0,' '' As she translates from one language to an...,' '' As she translates from one language to an...
1,' '' BDSM is solely based on consensual activi...,' '' BDSM is solely based on consensual activi...
2,' '' Critics criticized the use of the dispute...,' '' Critics criticized the use of the dispute...
3,' '' Do Re Mi '' ' is a song by Kurt Cobain of...,"' '' Do Re Mi '' ' is a song by Kurt Cobain , ..."
4,"' '' For Robert Price '' docetism '' , togethe...","' '' For Robert Price '' docetism '' , togethe..."


In [73]:
wikiDF_validation = pd.read_csv('/kaggle/input/wikisplit2/wikisplit2/wiki-split-master/validation.tsv', sep='\t',names=columns)
wikiDF_validation=wikiDF_validation.head(1000)
wikiDF_validation.head()

,unsplitted,splitted
0,"' Lauren Rose Crace ' , born 25th of May 1986 ...","' Lauren Rose Crace ' , born 25th of May 1986 ..."
1,'' A Living Library '' was Bonnie Sherk 's wor...,'' A Living Library '' was Sherk 's work that ...
2,"'' All Singing , All Dancing '' is the elevent...","'' All Singing , All Dancing '' is the elevent..."
3,'' Already Gone '' is a mid-tempo ballad in wh...,'' Already Gone '' is a mid-tempo ballad set i...
4,'' Bafana Bafana '' is a nickname given to the...,'' Bafana Bafana '' is a nickname given to the...


In [125]:
wikiDF_test = pd.read_csv('/kaggle/input/wikisplit2/wikisplit2/wiki-split-master/test.tsv', sep='\t',names=columns)
wikiDF_test.head()

,unsplitted,splitted
0,"' Bandolier - Budgie ' , a free iTunes app for...","' Bandolier - Budgie ' , a free iTunes app for..."
1,' Eden Black ' was grown from seed in the late...,' Eden Black ' was grown from seed in the late...
2,' Wilson should extend his stint on The Voice ...,' Wilson should extend his stint on The Voice ...
3,'' '' New York Mining Disaster 1941 '' '' was ...,'' '' New York Mining Disaster 1941 '' '' was ...
4,"'' ADAPTOGENS : Herbs for Strength , Stamina ,...","'' ADAPTOGENS : Herbs for Strength , Stamina ,..."


In [126]:
def cleaning(wikiDF):
    
    wikiDF['cleaned_title']=wikiDF['unsplitted'].apply(custom_standardization)
    wikiDF['cleaned_title']=wikiDF['cleaned_title'].apply(remove_alphanumeric)
    wikiDF['cleaned_title']=wikiDF['cleaned_title'].apply(remove_extra_spaces)
    
    return wikiDF
wikiDF = cleaning(wikiDF)
wikiDF_test = cleaning(wikiDF_test)
wikiDF_validation = cleaning(wikiDF_validation)

In [76]:
wikiDF.head()
wikiDF.iloc[0,2]

'As she translates from one language to another she tries to find the appropriate wording and context in English that would correspond to the work in Spanish her poems and stories started to have differing meanings in their respective languages'

In [127]:
cleanedwikiDF = wikiDF.drop(columns=columns)
cleanedwikiDF_test = wikiDF_test.drop(columns=columns)
cleanedwikiDF_validation = wikiDF_validation.drop(columns=columns)

#save the cleaned data
cleanedwikiDF.to_csv('cleaned_wiki.csv',index=False)
loadedwikiDF=pd.read_csv('/kaggle/working/cleaned_wiki.csv')
loadedwikiDF.head()

In [128]:
def convertTolist(loadedwikiDF):
    # convert sentence to list of words
    loadedwikiDF['cleaned_title_list']= loadedwikiDF['cleaned_title'].apply(lambda x: x.split(' '))
    # remove whitespaces and empty elements
    loadedwikiDF['cleaned_title_list']=loadedwikiDF['cleaned_title_list'].apply(lambda x: list(filter(None, x)))
#     return loadedwikiDF

convertTolist(loadedwikiDF)
convertTolist(cleanedwikiDF_test)
convertTolist(cleanedwikiDF_validation)

In [79]:
loadedwikiDF.head()

,cleaned_title,cleaned_title_list
0,As she translates from one language to another...,"[As, she, translates, from, one, language, to,..."
1,BDSM is solely based on consensual activities ...,"[BDSM, is, solely, based, on, consensual, acti..."
2,Critics criticized the use of the disputed fig...,"[Critics, criticized, the, use, of, the, dispu..."
3,Do Re Mi is a song by Kurt Cobain of the band ...,"[Do, Re, Mi, is, a, song, by, Kurt, Cobain, of..."
4,For Robert Price docetism together with encrat...,"[For, Robert, Price, docetism, together, with,..."


In [80]:
cleanedwikiDF_test.head()

,cleaned_title,cleaned_title_list
0,Bandolier Budgie a free iTunes app for iPad iP...,"[Bandolier, Budgie, a, free, iTunes, app, for,..."
1,Eden Black was grown from seed in the late by ...,"[Eden, Black, was, grown, from, seed, in, the,..."
2,Wilson should extend his stint on The Voice to...,"[Wilson, should, extend, his, stint, on, The, ..."
3,New York Mining Disaster was the second EP rel...,"[New, York, Mining, Disaster, was, the, second..."
4,ADAPTOGENS Herbs for Strength Stamina and Stre...,"[ADAPTOGENS, Herbs, for, Strength, Stamina, an..."


## Some functions to generate spelling mistakes.

In [81]:
# Replace random word's letter to a one of the 26 letters.

def replacementOfCharacters(text):
    Characterlen=len(text)
    randomIndex=random.randint(0,Characterlen-1)
    randomCharacter=random.choice(string.ascii_letters[:26])
    
    text=text[:randomIndex]+randomCharacter+text[randomIndex+1:]
    return text
replacementOfCharacters('hello')

'oello'

In [82]:
def SwappingCharacters(text):
    Characterlen=len(text)
    while(True):
        # Choose 2 random indices
        randomIndex=random.randint(0,Characterlen-1)
        randomIndex2=random.randint(randomIndex-1,randomIndex+1)
        
        # if the 2 indices are not the same -> break
        if(randomIndex!=randomIndex2 and randomIndex2<=Characterlen-1 and randomIndex2>=0):
            swap1=text[randomIndex]
            swap2=text[randomIndex2]
            if(swap1!=swap2):
                break

    # swap the text in the random index with the random index2

    textlist=list(text)
    textlist[randomIndex]=swap2
    textlist[randomIndex2]=swap1
    text=''.join(textlist)
    return text
    # return text
SwappingCharacters('hell0world')

'ehll0world'

In [83]:
def deleteRandomChar(text):
    Characterlen=len(text)
    randomIndex=random.randint(0,Characterlen-1)
    text=text[:randomIndex]+text[randomIndex+1:]
    return text
deleteRandomChar('hello')

'hllo'

In [84]:
def insertRandomChar(text):
    Characterlen=len(text)
    randomIndex=random.randint(0,Characterlen)
    randomCharacter=random.choice(string.ascii_letters[:26])
    text=text[:randomIndex]+randomCharacter+text[randomIndex:]
    return text
insertRandomChar('hello')

'khello'

In [124]:
test_data.head()

,missedup_abstract,cleaned_title,corrected_title
0,Bandolier Budgie a free iTunes app for Pad iPh...,Bandolier Budgie a free iTunes app for iPad iP...,Bandolier Budgie a free iTunes app for Play iP...
1,Eden Black was grown from seed in te late by S...,Eden Black was grown from seed in the late by ...,Eden Black was grown from seed in the late by ...
2,Wilson should extend his stint on The Voce to ...,Wilson should extend his stint on The Voice to...,Wilson should extend his stint on The Voce to ...
3,New York Mining Disaster was the second P rele...,New York Mining Disaster was the second EP rel...,New York Mining Disaster was the second EP rel...
4,ADAPTOGENS Herbs for Strength Stamina and Stre...,ADAPTOGENS Herbs for Strength Stamina and Stre...,ADAPTOGENS Herbs for Strength Stamina and Stre...


In [129]:
missedupsplitted=loadedwikiDF.copy()
test_data = cleanedwikiDF_test.copy()
validation_data = cleanedwikiDF_validation.copy()

def truncateList(textlist, N):
    if len(textlist)>N:
        textlist=textlist[:N]
    return textlist
missedupsplitted['cleaned_title_list']=missedupsplitted['cleaned_title_list'].apply(lambda x: truncateList(x,15))
test_data['cleaned_title_list']=test_data['cleaned_title_list'].apply(lambda x: truncateList(x,15))
validation_data['cleaned_title_list']=validation_data['cleaned_title_list'].apply(lambda x: truncateList(x,15))

In [86]:
def missupDataAtrandom(listOfwordss):
    np.random.seed(42)
    listOfwords=listOfwordss.copy()
    numberOfmissingups=np.random.randint(0,min(4,len(listOfwords)))

    for i in range(numberOfmissingups+1):
        typeOfMiss=np.random.randint(0,4)
        indexTomiss=np.random.randint(0,len(listOfwordss))
        while len(listOfwords[indexTomiss])<=1:
            indexTomiss=np.random.randint(0,len(listOfwords))
        if len(listOfwords[indexTomiss])>1:
            if typeOfMiss==0:
                listOfwords[indexTomiss]= replacementOfCharacters(listOfwords[indexTomiss])
            elif typeOfMiss==1:
                continue
                listOfwords[indexTomiss]= SwappingCharacters(listOfwords[indexTomiss])
            elif typeOfMiss==2:
                listOfwords[indexTomiss]= deleteRandomChar(listOfwords[indexTomiss])
            else:
                listOfwords[indexTomiss]= insertRandomChar(listOfwords[indexTomiss])
    return listOfwords
    # return randomint
missupDataAtrandom(['hello','world','how','are','you'])


['hello', 'world', 'ho', 'are', 'ysolu']

In [130]:
missedupsplitted['missedup_abstract_list']=missedupsplitted['cleaned_title_list'].apply(missupDataAtrandom)
test_data['missedup_abstract_list']=test_data['cleaned_title_list'].apply(missupDataAtrandom)
validation_data['missedup_abstract_list']=validation_data['cleaned_title_list'].apply(missupDataAtrandom)

In [88]:
missedupsplitted.iloc[0,1]==missedupsplitted.iloc[0,2]
print(missedupsplitted.iloc[0,1])
print(missedupsplitted.iloc[0,2])

['As', 'she', 'translates', 'from', 'one', 'language', 'to', 'another', 'she', 'tries', 'to', 'find', 'the', 'appropriate', 'wording']
['As', 'she', 'translates', 'from', 'one', 'language', 'to', 'another', 'she', 'tries', 'o', 'find', 'vnthe', 'appropriate', 'wording']


In [89]:
def convertToString(listOfWords):
    return ' '.join(listOfWords)

def list2string(df):
    
    df.drop(columns=['cleaned_title'],inplace=True)
    df['missedup_abstract']=df['missedup_abstract_list'].apply(convertToString)
    df['cleaned_title']=df['cleaned_title_list'].apply(convertToString)
    df.drop(columns=['missedup_abstract_list','cleaned_title_list'],inplace=True)

In [131]:
list2string(missedupsplitted)
list2string(test_data)
list2string(validation_data)

missedupsplitted.head()

,missedup_abstract,cleaned_title
0,As she translates from one language to another...,As she translates from one language to another...
1,BDSM is solely based on consensual activities ...,BDSM is solely based on consensual activities ...
2,Critics criticized the use of the disputed fig...,Critics criticized the use of the disputed fig...
3,Do Re Mi is a song by Kurt Cobain of he band N...,Do Re Mi is a song by Kurt Cobain of the band ...
4,For Robert Price docetism together with encrat...,For Robert Price docetism together with encrat...


In [91]:
hfDF=Dataset.from_pandas(missedupsplitted)
hfDF

Dataset({
    features: ['missedup_abstract', 'cleaned_title'],
    num_rows: 10000
})

In [92]:
hf_val = Dataset.from_pandas(missedupsplitted)
hf_val

Dataset({
    features: ['missedup_abstract', 'cleaned_title'],
    num_rows: 10000
})

#### as tarket tokenizer explaination
#### This is because the languages you're translating between (Engish and German in this case) have 
#### different tokenization vocabularies. This implies that tokens will get tokenized 
#### differently. MarianMT models have seq2seq (encoder-decoder) architectures, and both the encoder and 
#### decoder each have their own embedding matrix. This means that the encoder will have an embedding vector 
#### for the token '▁doctor', whereas the decoder will learn an embedding vector for the token '▁do', an
#### embedding vector for the token 'ctor', etc.

#### Tokenization vocabularies are typically built per language (although models like BLOOM just have one 
#### large vocabulary for all language tokens).

In [93]:
model_checkpoint='facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [94]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [0, 31414, 6, 42, 16, 10, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [96]:
max_input_length=100
max_target_length=100

def transProcessing(data):
    inputs=data['missedup_abstract']
    
    model_inputs = tokenizer(inputs, max_length=max_input_length , truncation=True)
    labels = tokenizer(data['cleaned_title'], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [97]:
sampleinputs=transProcessing(hfDF[:1])
print(sampleinputs['input_ids'])
print(sampleinputs['attention_mask'])
print(sampleinputs['labels'])

[[0, 1620, 79, 19303, 31, 65, 2777, 7, 277, 79, 5741, 1021, 465, 748, 3999, 700, 3901, 28776, 2]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[[0, 1620, 79, 19303, 31, 65, 2777, 7, 277, 79, 5741, 7, 465, 5, 3901, 28776, 2]]


In [98]:
tokenizedDS=hfDF.map(transProcessing,batched=True)
tokenizedDS

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['missedup_abstract', 'cleaned_title', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [99]:
tokenizedVal=hf_val.map(transProcessing,batched=True)
tokenizedVal

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['missedup_abstract', 'cleaned_title', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [100]:
from transformers import TFAutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,TFTrainingArguments

model=TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
config=model.config

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [101]:
batch_size=24
learning_rate=5e-5
weight_decay=0.01
adam_epsilon=1e-8
num_train_epochs=5

In [102]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

In [103]:
train_dataset = model.prepare_tf_dataset(
    tokenizedDS,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)
val_dataset = model.prepare_tf_dataset(
    tokenizedVal,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [104]:
from transformers import AdamWeightDecay
import tensorflow as tf
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer,metrics="accuracy",experimental_run_tf_function=False)

In [105]:
model.fit(train_dataset, validation_data=val_dataset, epochs=num_train_epochs)

Epoch 1/5
416/416 [==============================] - 264s 465ms/step - loss: 0.4084 - accuracy: 0.7219 - val_loss: 0.1479 - val_accuracy: 0.7562
Epoch 2/5
416/416 [==============================] - 177s 427ms/step - loss: 0.2144 - accuracy: 0.7454 - val_loss: 0.0669 - val_accuracy: 0.7695
Epoch 3/5
416/416 [==============================] - 178s 428ms/step - loss: 0.1385 - accuracy: 0.7573 - val_loss: 0.0343 - val_accuracy: 0.7741
Epoch 4/5
416/416 [==============================] - 178s 428ms/step - loss: 0.1369 - accuracy: 0.7571 - val_loss: 0.0214 - val_accuracy: 0.7780
Epoch 5/5
416/416 [==============================] - 179s 430ms/step - loss: 0.0725 - accuracy: 0.7679 - val_loss: 0.0189 - val_accuracy: 0.7801


In [106]:
model.fit(train_dataset, validation_data=val_dataset, epochs=num_train_epochs)

Epoch 1/5
416/416 [==============================] - 178s 428ms/step - loss: 0.0519 - accuracy: 0.7707 - val_loss: 0.0049 - val_accuracy: 0.7830
Epoch 2/5
416/416 [==============================] - 178s 428ms/step - loss: 0.0288 - accuracy: 0.7769 - val_loss: 0.0035 - val_accuracy: 0.7812
Epoch 3/5
416/416 [==============================] - 178s 428ms/step - loss: 0.0259 - accuracy: 0.7777 - val_loss: 0.0022 - val_accuracy: 0.7826
Epoch 4/5
416/416 [==============================] - 179s 430ms/step - loss: 0.0191 - accuracy: 0.7787 - val_loss: 0.0023 - val_accuracy: 0.7823
Epoch 5/5
416/416 [==============================] - 178s 428ms/step - loss: 0.0148 - accuracy: 0.7808 - val_loss: 0.0034 - val_accuracy: 0.7813


In [170]:
sentence='As she translates frmo her lanuage to anoher'

inputs = tokenizer(sentence, max_length=512, return_tensors='tf', padding="max_length", truncation=True)
predicted_abstract = model.generate(**inputs)
predicted_abstract
decoded=tokenizer.decode(predicted_abstract[0], skip_special_tokens=True)
decoded

'As she translates from her language to another'

## Model Evaluation

In [133]:
test_data=test_data.head(300)
print( test_data.shape)
test_data.head()

(300, 2)


,missedup_abstract,cleaned_title
0,Bandolier Budgie a free iTunes app for iPad iP...,Bandolier Budgie a free iTunes app for iPad iP...
1,Eden Black was grown from seed in the late by ...,Eden Black was grown from seed in the late by ...
2,Wilson should extend his stint on The Voice to...,Wilson should extend his stint on The Voice to...
3,New York Mining Disaster was the second EP rel...,New York Mining Disaster was the second EP rel...
4,ADAPTOGENS Herbs for Strength Stkmina and Stre...,ADAPTOGENS Herbs for Strength Stamina and Stre...


In [134]:
corrected_sentences = []
for sentence in test_data['missedup_abstract']:
    
    inputs = tokenizer(sentence, max_length=512, return_tensors='tf', padding="max_length", truncation=True)
    predicted_abstract = model.generate(**inputs)
    decoded=tokenizer.decode(predicted_abstract[0], skip_special_tokens=True)
    corrected_sentences.append(decoded)

test_data['corrected_title'] = corrected_sentences

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [116]:
test_data.head()

,missedup_abstract,cleaned_title,corrected_title
0,Bandolier Budgie a free iTunes app for Pad iPh...,Bandolier Budgie a free iTunes app for iPad iP...,Bandolier Budgie a free iTunes app for Play iP...
1,Eden Black was grown from seed in te late by S...,Eden Black was grown from seed in the late by ...,Eden Black was grown from seed in the late by ...
2,Wilson should extend his stint on The Voce to ...,Wilson should extend his stint on The Voice to...,Wilson should extend his stint on The Voce to ...
3,New York Mining Disaster was the second P rele...,New York Mining Disaster was the second EP rel...,New York Mining Disaster was the second EP rel...
4,ADAPTOGENS Herbs for Strength Stamina and Stre...,ADAPTOGENS Herbs for Strength Stamina and Stre...,ADAPTOGENS Herbs for Strength Stamina and Stre...


In [151]:
print((test_data.iloc[1,0]).split())
print((test_data.iloc[1,1]).split())
print(test_data.iloc[1,2].split())

['Eden', 'Black', 'was', 'grown', 'from', 'seed', 'in', 'the', 'late', 'by', 'Stepen', 'Morley', 'unndaer', 'his', 'conditions']
['Eden', 'Black', 'was', 'grown', 'from', 'seed', 'in', 'the', 'late', 'by', 'Stephen', 'Morley', 'under', 'his', 'conditions']
['Eden', 'Black', 'was', 'grown', 'from', 'seed', 'in', 'the', 'late', 'by', 'Stephen', 'Morley', 'under', 'his', 'conditions']


In [ ]:
print((test_data.iloc[0,0]).split())

In [141]:
test_data.head()

,missedup_abstract,cleaned_title,corrected_title
0,Bandolier Budgie a free iTunes app for iPad iP...,Bandolier Budgie a free iTunes app for iPad iP...,Bandolier Budgie a free iTunes app for iPad iP...
1,Eden Black was grown from seed in the late by ...,Eden Black was grown from seed in the late by ...,Eden Black was grown from seed in the late by ...
2,Wilson should extend his stint on The Voice to...,Wilson should extend his stint on The Voice to...,Wilson should extend his stint on The Voice to...
3,New York Mining Disaster was the second EP rel...,New York Mining Disaster was the second EP rel...,New York Mining Disaster was the second EP rel...
4,ADAPTOGENS Herbs for Strength Stkmina and Stre...,ADAPTOGENS Herbs for Strength Stamina and Stre...,ADAPTOGENS Herbs for Strength Strikmina and St...


#### Note that in the above sentences no miscorrected words ,but word accuracy will not consider it

# Metrics
## 1- Word Accuracy
### Measure how much the model able to correct the misspelled-up words


In [139]:
# def get_n_different_words(test_data,target_column2_index):
i = -1
missed_words = []
mistakes = 0
for sentence in test_data["missedup_abstract"]:
    i+= 1
    for word in sentence.split():
        if word not in test_data.iloc[i,1]:
            missed_words.append(word)
n_missed = len(missed_words)
print("The number of miscorrected words =",n_missed)

i = -1
for sentence in test_data["corrected_title"]:
    i+= 1
    for word in sentence.split():
        if word in missed_words:
            mistakes +=1
accuracy = (n_missed - mistakes) / n_missed
print("The model accuracy is : ",round(accuracy,2)*100 ,'%' )

The number of miscorrected words = 446
The model accuracy is :  90.0 %


### 2- BLEU 

In [143]:
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import matthews_corrcoef

bleu_scores = 0
for reference , corrected in zip(test_data['cleaned_title'],test_data['corrected_title']):
    bleu_scores += sentence_bleu(reference,corrected)
    
bleu_score = bleu_scores /300
print("BLEU Score:", bleu_score)

BLEU Score: 0.7280664871927081


## 3- MCC
## Take the original sentnce as a reference to both misspelled-up and corrected data
### In misspelled-up sentence : we label the misspelled-up word as 0
### In corrected sentence : we label the word that is not catched as 0

In [145]:
from sklearn.metrics import matthews_corrcoef

mcc_scores = 0
for reference, corrected,wrong in zip(test_data['cleaned_title'], test_data['corrected_title'],test_data['missedup_abstract']):
    corrected_labels = [1 if word in corrected.split() else 0 for word in reference.split()]
    missedup_labels = [1 if word in wrong.split() else 0 for word in reference.split()]
    mcc_scores += matthews_corrcoef(corrected_labels, missedup_labels)
print("Average Matthews Correlation Coefficient:", mcc_scores/300)

Average Matthews Correlation Coefficient: 0.3881404213443621


## 4- Rouge 

In [163]:
from seq_eval import SeqEval

evaluator = SeqEval()

references =test_data['cleaned_title']
candidates =test_data['corrected_title']

scores = evaluator.evaluate(candidates, references, verbose=True)

***************
* BLEU SCORES *
***************
BLEU-1:  0.8943610758999911
BLEU-2:  0.8672182457360892
BLEU-3:  0.8437794731977121
BLEU-4:  0.822096478695048


****************
* ROUGE SCORES *
****************
ROUGE-1 PRECISION:  0.9435336330336332
ROUGE-1 RECALL:  0.9036301846301844
ROUGE-1 F1 :  0.9210807070579473


ROUGE-2 PRECISION:  0.8872508325008327
ROUGE-2 RECALL:  0.8445231065231072
ROUGE-2 F1 :  0.8629029693422106


ROUGE-L PRECISION:  0.9432955377955379
ROUGE-L RECALL:  0.9034079624079622
ROUGE-L F1 :  0.9208508220004762


*********************
* DISTINCT-N SCORES *
*********************
INTER DIST-1:  0.0021672459610847902
INTER DIST-2:  0.03129786792328803
INTRA DIST-1:  0.2852984351089419
INTRA DIST-2:  0.780636163324041




Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

******************************************************
* SEMANTIC TEXTUAL SIMILARITY (Sentence Transformer) *
******************************************************
COSINE SIMILARITY:  0.9388093058268229


In [156]:
!pip install sequence-evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
